<a href="https://colab.research.google.com/github/Saybuuu/test_cases/blob/main/sub_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

21vek_query_classification_path = kagglehub.competition_download('21vek-query-classification')
vladislavbukhtoyarov_dataset_path = kagglehub.dataset_download('vladislavbukhtoyarov/dataset')
vladislavbukhtoyarov_kaggle_api_key_path = kagglehub.dataset_download('vladislavbukhtoyarov/kaggle-api-key')

print('Data source import complete.')


In [ ]:
import pandas as pd
import os

In [ ]:
category_path = "/kaggle/input/21vek-query-classification/categories.csv"
sample_path   = "/kaggle/input/21vek-query-classification/sample_submission.csv"
test_path     = "/kaggle/input/21vek-query-classification/test.csv"
train_path    = "/kaggle/input/21vek-query-classification/train.csv"

In [ ]:
test = pd.read_csv(test_path)
train = pd.read_csv(train_path)
del train['ID']
sample = pd.read_csv(sample_path)
categories = pd.read_csv(category_path)

In [ ]:
print(train.isnull().sum())

In [ ]:
# categories.info
print(len(categories))

In [ ]:
used_ids = sorted(train['CategoryID'].unique())
print("Missing:", [x for x in range(min(used_ids), max(used_ids)+1) if x not in used_ids])

In [ ]:
print(categories[categories['CategoryID'] == 62])

In [ ]:
vinyl_products = [
    {"CategoryID": 62, "Query": "Виниловая пластинка The Beatles - Abbey Road"},
    {"CategoryID": 62, "Query": "Винил Pink Floyd - The Dark Side of the Moon"},
    {"CategoryID": 62, "Query": "Виниловая пластинка Led Zeppelin - IV"},
    {"CategoryID": 62, "Query": "Винил Queen - A Night at the Opera"},
    {"CategoryID": 62, "Query": "Виниловая пластинка The Rolling Stones - Sticky Fingers"},
    {"CategoryID": 62, "Query": "Пластинка Nirvana"},
    {"CategoryID": 62, "Query": "Виниловая пластинка Michael Jackson - Thriller"},
    {"CategoryID": 62, "Query": "Виниловая пластинка David Bowie - Heroes"},
    {"CategoryID": 62, "Query": "Виниловая пластинка Radiohead - OK Computer"},
    {"CategoryID": 62, "Query": "Винил Master of Puppets"},
    {"CategoryID": 62, "Query": "Виниловая пластинка The Who - Who's Next"},
    {"CategoryID": 62, "Query": "Виниловая пластинка U2 "},
    {"CategoryID": 62, "Query": "Red Hot Chili Peppers - Californication"},
    {"CategoryID": 62, "Query": "пластинка AC/DC - Back in Black"},
    {"CategoryID": 62, "Query": "Виниловая пластинка Pearl Jam - Ten"},
    {"CategoryID": 62, "Query": " пластинка Oasis - (What's the Story) Morning Glory?"},
    {"CategoryID": 62, "Query": "Виниловая пластинка Guns N' Roses - Appetite for Destruction"},
    {"CategoryID": 62, "Query": "пластинка The Doors - The Doors"},
    {"CategoryID": 62, "Query": "пластинка Bob Dylan - Highway 61 Revisited"},
    {"CategoryID": 62, "Query": "пластинка Simon & Garfunkel - Bridge Over Troubled Water"},
]

train = pd.concat([train, pd.DataFrame(vinyl_products)], ignore_index=True)
print(len(train))

In [ ]:
counts = train.groupby('CategoryID').size()
counts = counts.sort_values()
counts.head(10)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 5))
train.groupby('CategoryID').Query.count().sort_values().plot.bar(x='CategoryID', y='Query')
plt.xticks(rotation=90)
plt.show()

In [ ]:
!pip install pymorphy3

In [ ]:
!pip install googletrans

In [ ]:
from pymorphy3 import MorphAnalyzer
import nltk
import random
from sklearn.utils import resample
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from transformers import pipeline
import torch
import math
from googletrans import Translator
import time
import asyncio

In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.get_device_capability())

In [ ]:
categories.rename(columns={'CategoryName': 'Query'}, inplace=True)
train = pd.concat([train, categories], ignore_index=True)
print(len(train))

In [ ]:
def char_augment(text, prob=0.1):
    result = []
    for char in text:
        if random.random() < prob:
            if char.isalpha():
                if random.random() > 0.5:
                    result.append(char.swapcase())
                else:
                    key_ru = {'о':'а', 'е':'э', 'ш':'щ', '0':'1'}
                    key_en = {'o':'p', 'e':'r', 't':'y', '1':'2'}
                    replacement = key_ru.get(char.lower(), key_en.get(char.lower(), char))
                    result.append(replacement)
            elif char.isdigit():
                result.append(str((int(char) + random.randint(0, 2)) % 10))
            else:
                result.append(char)
        else:
            result.append(char)
    return ''.join(result)

In [ ]:
nltk.download('wordnet')

In [ ]:
morph_ru = MorphAnalyzer()

In [ ]:
def synonym_replacement(text, lang='ru', prob=0.3):
    words = text.split()
    for i, word in enumerate(words):
        if random.random() < prob:
            if lang == 'ru':
                parsed = morph_ru.parse(word)[0]
                if parsed.score > 0.5 and len(parsed.lexeme) > 1:
                    words[i] = random.choice(parsed.lexeme).word
            else:  # en
                syns = wn.synsets(word)
                if syns:
                    lemmas = [l.name() for syn in syns for l in syn.lemmas()]
                    if lemmas:
                        words[i] = random.choice(lemmas)
    return ' '.join(words)

In [ ]:
nltk.download('stopwords')

In [ ]:
stopwords_ru = set(stopwords.words('russian'))
stopwords_en = set(stopwords.words('english'))

In [ ]:
def shuffle_words(text, prob=0.3):
    words = text.split()
    protected_indices = [
        idx for idx, word in enumerate(words) if any(char.isdigit() for char in word)
    ]
    non_protected_indices = [
        idx for idx in range(len(words)) if idx not in protected_indices
    ]
    for i in non_protected_indices:
        if random.random() < prob:
            j = random.choice(non_protected_indices)
            words[i], words[j] = words[j], words[i]

    return ' '.join(words)

In [ ]:
async def back_translate(text, src_lang='auto', intermediate_langs=['de', 'fr']):

    translator = Translator()

    try:
        if src_lang == 'auto':
            detection = await translator.detect(text)
            detected_lang = detection.lang
            src_lang = detected_lang if detected_lang in ['en', 'ru'] else 'ru'

        intermediate_lang = random.choice(intermediate_langs)

        translated_result = await translator.translate(text, dest=intermediate_lang, src=src_lang)
        translated_text = translated_result.text

        await asyncio.sleep(0.5)

        back_translated_result = await translator.translate(
            translated_text, dest=src_lang, src=intermediate_lang
        )
        back_translated_text = back_translated_result.text

        return back_translated_text
    except Exception as e:
        print(f"Error in back_translate: {e}")
        return text


In [ ]:
new_rows = []
for row in categories.itertuples():
    new_query = await back_translate(row.Query)
    new_rows.append(
        {
            'CategoryID': int(row.CategoryID),
            'Query': f'{new_query}'
        }
    )

train = pd.concat([train, pd.DataFrame(new_rows)], ignore_index=True)
print(len(train))

In [ ]:
counts = train.groupby('CategoryID').size()
counts = counts.sort_values()
counts.head(10)

In [ ]:
def text_mixing(text1, text2, split_range=(0.3, 0.7)):
    words1 = text1.split()
    words2 = text2.split()

    split_point1 = int(len(words1) * random.uniform(*split_range))
    split_point2 = int(len(words2) * random.uniform(*split_range))

    mixed_words = words1[:split_point1] + words2[split_point2:]
    mixed_text = ' '.join(mixed_words)

    return mixed_text

In [ ]:
for category_id in counts.index:
    new_rows = []

    if 2 <= counts[category_id] < 50:
        filtered_queries = train[train['CategoryID'] == category_id]
        iters = int(30 - counts[category_id])
        for row in filtered_queries.itertuples():
            queries = filtered_queries[filtered_queries.index != row.Index]['Query'].tolist()
            new_query = text_mixing(row.Query, random.choice(queries))
            new_rows.append(
                {
                    'CategoryID': int(row.CategoryID),
                    'Query': f'{new_query}'
                }
            )
            iters -=1
        if iters == 0:
            break

        train = pd.concat([train, pd.DataFrame(new_rows)], ignore_index=True)

print(len(train))

In [ ]:
counts = train.groupby('CategoryID').size()
counts = counts.sort_values()
counts.head(10)

In [ ]:
for category_id in counts.index:
    new_rows = []
    filtered_queries = train[train['CategoryID'] == category_id]

    if counts[category_id] < 50:
        for row in filtered_queries.itertuples():

            new_rows.append(
                {
                    'CategoryID': category_id,
                    'Query': f'{synonym_replacement(str(row.Query))}'
                }
            )


    elif counts[category_id] < 150:
        iters = int((150 - counts[category_id]) / 2)

        for row in filtered_queries.itertuples():
            new_rows.append(
                {
                    'CategoryID': category_id,
                    'Query': f'{synonym_replacement(str(row.Query))}'
                }
            )
            iters -=1

            if iters == 0:
                break


    if counts[category_id] < 350:
        iters = int((350 - counts[category_id]) / 3)

        for row in filtered_queries.itertuples():
            new_rows.append(
                {
                    'CategoryID': category_id,
                    'Query': f'{synonym_replacement(str(row.Query))}'
                }
            )
            iters -=1

            if iters == 0:
                break


    train = pd.concat([train, pd.DataFrame(new_rows)], ignore_index=True)


In [ ]:
print(len(train))

In [ ]:
for category_id in counts.index:
    new_rows = []
    filtered_queries = train[train['CategoryID'] == category_id]
    if counts[category_id] < 500:
        iters = int((500 - counts[category_id]) / 2)

        for row in filtered_queries.itertuples():
            query = str(row.Query)
            new_rows.append(
                {"CategoryID": category_id, "Query": f"{shuffle_words(query)}"}
            )
            iters -=1

            if iters == 0:
                break

    elif 500 <= counts[category_id] < 800:
        iters = int((800 - counts[category_id]) / 4)

        for row in filtered_queries.itertuples():
            query = str(row.Query)
            iters = int((150 - counts[category_id]) / 2)
            new_rows.append(
                {"CategoryID": category_id, "Query": f"{shuffle_words(query)}"}
            )
            iters -=1

            if iters == 0:
                break

    train = pd.concat([train, pd.DataFrame(new_rows)], ignore_index=True)

print(len(train))

In [ ]:
for category_id in counts.index:
    new_rows = []
    filtered_queries = train[train['CategoryID'] == category_id]
    iters = int(counts[category_id] / 10)

    for row in filtered_queries.itertuples():
        query = char_augment(row.Query)
        new_rows.append(
            {
                "CategoryID": category_id,
                "Query": f"{query}"
            }
        )
        iters -=1

        if iters == 0:
            break

    train = pd.concat([train, pd.DataFrame(new_rows)], ignore_index=True)

print(len(train))

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 5))
train.groupby('CategoryID').Query.count().sort_values().plot.bar(x='CategoryID', y='Query')
plt.xticks(rotation=90)
plt.show()

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train['Query'].tolist(), train['CategoryID'].tolist(), test_size=0.2, random_state=42
)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

In [ ]:
class QueryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = QueryDataset(train_encodings, train_labels)
val_dataset = QueryDataset(val_encodings, val_labels)


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(train['CategoryID'].unique()),
                                                      ignore_mismatched_sizes=True)

In [ ]:
from torch.utils.data import DataLoader
from torch.optim import AdamW

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
train_losses = []
val_losses = []
f1_scores = []
precisions = []
recalls = []

In [ ]:
device = torch.device("cuda")
model.to(device)

In [ ]:
from tqdm import tqdm

In [ ]:
learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)

epochs = 5

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc="Training", leave=False):
        optimizer.zero_grad()

        batch = {key: val.to(device) for key, val in batch.items()}
        outputs = model(**batch)

        loss = loss_fn(outputs.logits, batch["labels"])
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    train_losses.append(total_loss / len(train_loader))

    model.eval()
    y_true, y_pred = [], []
    val_loss = 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation", leave=False):
            batch = {key: val.to(device) for key, val in batch.items()}
            outputs = model(**batch)

            predictions = torch.argmax(outputs.logits, dim=1)
            y_true.extend(batch["labels"].cpu().numpy())
            y_pred.extend(predictions.cpu().numpy())

            val_loss += loss_fn(outputs.logits, batch["labels"]).item()

    val_losses.append(val_loss / len(val_loader))

    f1 = f1_score(y_true, y_pred, average="weighted")
    precision = precision_score(y_true, y_pred, average="weighted")
    recall = recall_score(y_true, y_pred, average="weighted")

    f1_scores.append(f1)
    precisions.append(precision)
    recalls.append(recall)

    print(f"Epoch {epoch + 1}: Loss = {train_losses[-1]:.4f}, F1 = {f1:.4f}, Precision = {precision:.4f}, Recall = {recall:.4f}")


In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range(1, epochs + 1), train_losses, label="Train Loss", marker="o")
plt.plot(range(1, epochs + 1), val_losses, label="Val Loss", marker="o")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training & Validation Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, epochs + 1), f1_scores, label="F1-score", marker="s")
plt.plot(range(1, epochs + 1), precisions, label="Precision", marker="^")
plt.plot(range(1, epochs + 1), recalls, label="Recall", marker="d")
plt.xlabel("Epoch")
plt.ylabel("Score")
plt.title("Evaluation Metrics")
plt.legend()

plt.show()


In [ ]:
def predict_category(query):
    model.eval()

    inputs = tokenizer(query, return_tensors="pt", truncation=True, padding=True, max_length=128)

    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    predicted_category = torch.argmax(outputs.logits, dim=1).item()

    return predicted_category

In [ ]:
test_df = test.copy()
test_df['CategoryID'] = test_df['Query'].apply(predict_category)
del test_df['Query']
test_df.head(5)

In [ ]:
test_df.to_csv('sub_file_3.csv', index=False)

In [ ]:
!mkdir -p ~/.kaggle
!cp /kaggle/input/kaggle-api-key/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions submit -c 21vek-query-classification -f sub_file_3.csv -m "bert_4_epoch"